In [ ]:
import json
import time
import geojson
import requests
import geopandas
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings('ignore')

Скачивание датасета "места общественного питания в городе Москва" при помощи API request с сайта data.mos.ru

In [ ]:
def get_all_data_by_API():
    data_clean = []
    url_count = 'https://apidata.mos.ru/v1/datasets/1903/count'
    params_count = {
        'api_key': '2142785399b8838f0312c6cd35ddc39b'
    }
    resp = requests.get(url=url_count, params=params_count)
    count = resp.text
    time.sleep(1)

    url_rows = 'https://apidata.mos.ru/v1/datasets/1903/rows'
    for i in range(0, int(count), 500):
        print(i)
        # skip = 0 if i == 500 else 500
        params = {
            'api_key': '2142785399b8838f0312c6cd35ddc39b',
            '$orderby': 'global_id',
            '$top': 500,
            '$skip': i
        }
        time.sleep(3)
        resp_rows = requests.get(url=url_rows, params=params)
        print(resp_rows)
        if (resp_rows.status_code == 200):
            data = resp_rows.json()

            def prepare_dict(response_item):
                temp = {
                    'Name': response_item['Cells']['Name'],
                    'IsNetObject': response_item['Cells']['IsNetObject'],
                    'OperatingCompany': response_item['Cells']['OperatingCompany'],
                    'AdmArea': response_item['Cells']['AdmArea'],
                    'District': response_item['Cells']['District'],
                    'Address': response_item['Cells']['Address'],
                    'PublicPhone': response_item['Cells']['PublicPhone'][0]['PublicPhone'] if response_item['Cells']['PublicPhone'][0]['PublicPhone'] != "" else "",
                    'SeatsCount': response_item['Cells']['SeatsCount'],
                    'SocialPrivileges': response_item['Cells']['SocialPrivileges'],
                    'Latitude' : response_item['Cells']['geoData']['coordinates'][1],
                    'Longitude' : response_item['Cells']['geoData']['coordinates'][0],
                }                
                return temp

            data_clean.extend([prepare_dict(item) for item in data if not isinstance(item, str)])
            print('legnth = ', len(data_clean))

    df = pd.DataFrame(data_clean)
    df.insert(loc=0, column='Number', value=range(1, len(df) + 1))
    return df, count

df, count = get_all_data_by_API()
print(df.count())

# записываем результат в csv
df.to_csv('./eating.csv')

In [ ]:
df = pd.read_csv('./eating.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df.head()

In [ ]:
df['IsNetObject'].unique()

In [ ]:
df['AdmArea'].unique()

In [ ]:
df['District'].unique()

In [ ]:
dfg=df[df['IsNetObject'] == 'нет'].groupby('AdmArea').count().reset_index()
dfg

Добавляем колонку рейтинга с дефолтным значением -1.

In [ ]:
df_search_google = df
df_search_google.insert(df_search_google.shape[1], 'Rating', -1)
df_search_google["Address"] = df_search_google["Address"].str.replace('Российская Федерация,', '').str.strip()
df_search_google['Address'] = df_search_google['Address'].apply(lambda x: (",").join(list(filter(lambda y: 'муниципальный округ' not in y, x.split(',')))))

Отправляем запросы с названием заведения и его адресом гуглу и парсим возвращаемый результат "оценка заведения"

In [ ]:
for index, row in df_search_google.iterrows():
    if 'школа' not in df_search_google.at[index, 'Name'].lower():
        search_place = row['Name'] + ' ' + row['Address']
        with requests.Session() as s:
            url = f"https://www.google.com/search?"
            headers = {
                "referer":"referer: https://www.google.com/",
                "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36",
            }
            params = {
                "q": search_place
            }

            response = s.get(url, headers=headers, params=params)
            soup = BeautifulSoup(response.text, 'html.parser')
        myspan = soup.find_all("span", {"class": "Aq14fc"})
        if myspan != []:
            print(float(myspan[0].text.replace(',','.')))
            print(index)
            print()
            df_search_google['Rating'][index] = float(myspan[0].text.replace(',','.'))

Сохраним датафрейм в новый датасет

In [ ]:
df_search_google.to_csv('eating_with_ratings.csv')

In [ ]:
from bs4 import BeautifulSoup
import requests


def get_google_img(query):
    """
    gets a link to the first google image search result
    :param query: search query string
    :result: url string to first result
    """
    url = "https://www.google.com/search?q=" + str(query) + "&source=lnms&tbm=isch"
    headers={'User-Agent':"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.134 Safari/537.36"}

    html = requests.get(url, headers=headers).text

    soup = BeautifulSoup(html, 'html.parser')
    print(soup)
    image = soup.find("img",{"class":"t0fcAb"})
    print(image)
    if not image:
        return 
    return image['src']


if __name__ == '__main__':
    query = "хачапури"
    print(get_google_img(query) or "no image found")

Тестирование отрисовки пути из одной точки в другую через osmnx and networkx

In [ ]:
import osmnx as ox
import networkx as nx

ox.config(log_console=True, use_cache=True)

start_latlng = (55.7765825, 37.6758596)
end_latlng = (55.776655763415, 37.674832)

place = 'Moscow, Russia'

mode = 'walk'
optimizer = 'time'

graph = ox.graph_from_place(place, network_type = mode)
orig_node = ox.get_nearest_node(graph, start_latlng)
dest_node = ox.get_nearest_node(graph, end_latlng)
shortest_route = nx.shortest_path(graph, orig_node,dest_node,
                                  weight=optimizer)

In [ ]:
shortest_route_map = ox.plot_route_folium(graph, shortest_route, 
                                          tiles='openstreetmap')
shortest_route_map

Тестирование "определение координат по адресу"

In [ ]:
entry = "https://nominatim.openstreetmap.org/search"
params = {
    'q': 'Москва, пушкинская, 7',
    'format': 'json',
    'polygon': 1,
    'addressdetails':1
}
r = requests.get(entry, params=params)
(r.json()[0])

Был скачен ao.geojson - геоданные об округах Москвы

In [ ]:
import geopandas as gpd
df_geo = gpd.read_file('ao.geojson')
df_geo

In [ ]:
df_geo.insert(df_geo.shape[1], 'centroid', df_geo.geometry.centroid)

In [ ]:
m = folium.Map(location=[55.751999, 37.617734], zoom_start=12)

for _, r in df_geo.iterrows():
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'yellow'})
    folium.Popup(r['NAME']).add_to(geo_j)
    geo_j.add_to(m)
    
for _, r in df_geo.iterrows():
    lat = r['centroid'].y
    lon = r['centroid'].x
    folium.Marker(location=[lat, lon],
                  popup='Округ: {}'.format(r['NAME'])).add_to(m)

m

Спарсим таблицу с доп информацией об округах Москвы с вики

In [ ]:
wikiurl = "https://ru.wikipedia.org/wiki/%D0%90%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%B8%D0%B2%D0%BD%D0%BE-%D1%82%D0%B5%D1%80%D1%80%D0%B8%D1%82%D0%BE%D1%80%D0%B8%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D0%B5_%D0%B4%D0%B5%D0%BB%D0%B5%D0%BD%D0%B8%D0%B5_%D0%9C%D0%BE%D1%81%D0%BA%D0%B2%D1%8B"
table_class = "standard sortable jquery-tablesorter"
response = requests.get(wikiurl)
print(response.status_code)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
wiki_res = soup.find('table',{'class':"standard"})
wiki_res

Составляем новый датафрейм из таблицы вики

In [ ]:
links = wiki_res.findAll('a')
clear_links = [link.text for link in links if 'title' in link.attrs]
df_okrug = pd.DataFrame()
df_okrug['Okrug'] = clear_links
df_okrug

In [ ]:
rows = indiatable.findAll("tr")
wiki_res = []
for row in rows[1:]:
    data = [d.text.rstrip() for d in row.find_all('td')]
    wiki_res.append(data)

header = [th.text.rstrip() for th in rows[0].find_all('th')][1:]
header

In [ ]:
data_1 = []
for row in rows[1:]:
    data = [d.text.rstrip() for d in row.select('td')]
    data_1.append(data)

In [ ]:
data_1 = pd.DataFrame(data_1, columns=header)
df_copy = data_1.copy()
df_copy.head(13)

In [ ]:
df_copy = df_copy[:-1]
df_copy

Итоговый датафрейм спарсенный с вики

In [ ]:
moscow_data = pd.concat([df_okrug, df_copy], axis=1)
moscow_data

In [ ]:
moscow_data.to_csv('moscow.csv')